In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR

In [2]:
file_path = r"datos_apartamentos_rent.xlsx"
df = pd.read_excel(file_path)
df = pd.DataFrame(df)
# Se crea una columna con el precio mensual
df["price_monthly"] = df.apply(lambda x: x["price"] * 4 if x["price_type"] == "Weekly" else x["price"], axis=1)

In [3]:
# Extraer las comodidades de los inmueblde de la columna "amenities"
def get_unique_amenities(df, column="amenities"):
    unique_amenities = set()
    df[column].dropna().apply(lambda x: unique_amenities.update(x.split(",")))
    return list(unique_amenities)

# Obtener lista sin duplicados
unique_amenities_list = get_unique_amenities(df)

# Tarea 3

In [4]:
# LIMPIEZA DE DATOS

# Se eliminan las filas con valores nulos en las columnas latitude y longitude
df = df.dropna(subset=['latitude', 'longitude'])

# Se detectó que los valores de latitude y longitude se importaron mal, se procede a corregir
df['latitude'] = df['latitude']/10000
df['longitude'] = df['longitude']/10000

# Se reemplazan los datos nulos de address por una respuesta negativa
df['address'].fillna('No adress given', inplace=True)

# Se reemplazan los valores nulos de la columna "pets_allowed" por "None" para que pandas no confunda None con null
df["pets_allowed"] = df["pets_allowed_corrected"].apply(lambda x: "None" if x == "No pets" else x)

C:\Users\felip\AppData\Local\Temp\ipykernel_2956\2019282392.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['address'].fillna('No adress given', inplace=True)


In [5]:
df['cityname_corrected'] = df['cityname']

df.loc[df['cityname'].isna() & (df['latitude'] == 39.8163), 'cityname_corrected'] = 'Lebanon'
df.loc[df['cityname'].isna() & (df['latitude'] == 28.45900), 'cityname_corrected'] = 'Trilby'

null_counts = df.isnull().sum()
print(null_counts)

id                           0
category                     0
title                        0
body                         0
amenities                 3542
bathrooms                   34
bedrooms                     7
currency                     0
fee                          0
has_photo                    0
pets_allowed              1748
price                        0
price_display                0
price_type                   0
square_feet                  0
address                      0
cityname                    67
state                       67
latitude                     0
longitude                    0
source                       0
time                         0
pets_allowed_corrected    1748
price_monthly                0
cityname_corrected           0
dtype: int64


In [6]:
# Se corrige el estado para los datos nulos
df['state_corrected'] = df['state']

df.loc[df['state'].isna() & (df['latitude'] == 39.8163), 'state_corrected'] = 'KS'
df.loc[df['state'].isna() & (df['latitude'] == 28.4590), 'state_corrected'] = 'FL'

#Se verifica que no quedan datos nulos

null_counts = df.isnull().sum()
print(null_counts)

id                           0
category                     0
title                        0
body                         0
amenities                 3542
bathrooms                   34
bedrooms                     7
currency                     0
fee                          0
has_photo                    0
pets_allowed              1748
price                        0
price_display                0
price_type                   0
square_feet                  0
address                      0
cityname                    67
state                       67
latitude                     0
longitude                    0
source                       0
time                         0
pets_allowed_corrected    1748
price_monthly                0
cityname_corrected           0
state_corrected              0
dtype: int64


In [7]:
# Se reemplazan los datos nulos de bedrooms por la media
df['bedrooms'].fillna(df['bedrooms'].mean(), inplace=True)

C:\Users\felip\AppData\Local\Temp\ipykernel_2956\3372822796.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['bedrooms'].fillna(df['bedrooms'].mean(), inplace=True)


In [8]:
# Se reemplazan los datos nulos de bathrooms por una regresión lineal (price_monthly, square_feet, bedrooms)

# Imputar bathrooms
bathrooms_notnull = df[df['bathrooms'].notnull()]
bathrooms_null = df[df['bathrooms'].isnull()]
features = ['bedrooms', 'square_feet', 'price_monthly']  # Variables predictoras

model_bathrooms = LinearRegression()
model_bathrooms.fit(bathrooms_notnull[features], bathrooms_notnull['bathrooms'])
df.loc[df['bathrooms'].isnull(), 'bathrooms'] = model_bathrooms.predict(bathrooms_null[features])

null_counts = df.isnull().sum()
print(null_counts)

id                           0
category                     0
title                        0
body                         0
amenities                 3542
bathrooms                    0
bedrooms                     0
currency                     0
fee                          0
has_photo                    0
pets_allowed              1748
price                        0
price_display                0
price_type                   0
square_feet                  0
address                      0
cityname                    67
state                       67
latitude                     0
longitude                    0
source                       0
time                         0
pets_allowed_corrected    1748
price_monthly                0
cityname_corrected           0
state_corrected              0
dtype: int64


In [9]:
# IMPORTANTE HACER PIP INSTALL SKLEARN SI NO FUNCIONAN LOS IMPORTS

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor

# Mapear valores de pets_allowed a números
pets_mapping = {
    "None": 0,
    "Cats": 1,
    "Dogs": 2,
    "Cats,Dogs": 3
}
df['pets_allowed'] = df['pets_allowed'].map(pets_mapping)

categorical_features = ['category', 'source']

# Se hace la codificación de las variables categóricas
df_encoded = pd.get_dummies(df, columns=categorical_features, drop_first=True)

# Seleccionar las variables predictoras
pets_features = ['bedrooms', 'square_feet', 'price_monthly'] + [col for col in df_encoded.columns if col.startswith('category_') or col.startswith('source_')]

In [10]:
# Aplicar imputación múltiple con IterativeImputer usando RandomForest
imputer = IterativeImputer(estimator=RandomForestRegressor(n_estimators=100, random_state=42), max_iter=10)

# Aplicar imputación asegurando que la salida sea un DataFrame
imputed_values = imputer.fit_transform(df_encoded[['pets_allowed'] + pets_features])

In [11]:
# Convertir el array imputado nuevamente a un DataFrame
df_imputed = pd.DataFrame(imputed_values, columns=['pets_allowed'] + pets_features, index=df_encoded.index)

# Redondear los valores imputados de pets_allowed y convertir a enteros
df_imputed['pets_allowed'] = df_imputed['pets_allowed'].round().astype(int)

# Sobrescribir los valores imputados en el df original
df['pets_allowed'] = df_imputed['pets_allowed']

# Devolver el mapeo
reverse_mapping = {0: "None", 1: "Cats", 2: "Dogs", 3: "Cats,Dogs"}
df['pets_allowed'] = df['pets_allowed'].map(reverse_mapping)

df['pets_allowed'].value_counts()

pets_allowed
Cats,Dogs    6580
None         2414
Cats          567
Dogs          429
Name: count, dtype: int64

In [12]:
# Se corrigen los valores nulos de amenities mediante el texto de la columna body
# Función para extraer amenities de "body"
def extract_amenities_from_body(body_text, amenities_list):
    if pd.isna(body_text):  # Si el texto es NaN, devolver None
        return None
    found_amenities = [amenity for amenity in amenities_list if amenity.lower() in body_text.lower()] #.lower() devuelve todo el texto en minusc.
    return ",".join(found_amenities) if found_amenities else None #Unir las amenities como la columna original

In [13]:
# Funcion para completar los valores nulos en "amenities"
def completar_amenities(row):
    if pd.isna(row["amenities"]):  # Si el valor en "amenities" es NaN se extrae los amenities de "body"
        return extract_amenities_from_body(row["body"], unique_amenities_list)
    return row["amenities"]

# Aplicar la función a cada fila
df["amenities"] = df.apply(completar_amenities, axis=1)

# Reemplazar los valores nulos restantes en "amenities" por "No Amenities"
df["amenities"].fillna("No Amenities", inplace=True)

null_counts = df.isnull().sum()
print(null_counts)

id                           0
category                     0
title                        0
body                         0
amenities                    0
bathrooms                    0
bedrooms                     0
currency                     0
fee                          0
has_photo                    0
pets_allowed                 0
price                        0
price_display                0
price_type                   0
square_feet                  0
address                      0
cityname                    67
state                       67
latitude                     0
longitude                    0
source                       0
time                         0
pets_allowed_corrected    1748
price_monthly                0
cityname_corrected           0
state_corrected              0
dtype: int64


C:\Users\felip\AppData\Local\Temp\ipykernel_2956\1216768869.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["amenities"].fillna("No Amenities", inplace=True)


In [14]:
# Codificación de las variables categóricas
categorical_features = ['category', 'pets_allowed', 'cityname_corrected', 'state_corrected', 'source']
df = pd.get_dummies(df, columns=categorical_features, drop_first=True)

df['year'] = pd.to_datetime(df['time'], unit='s').dt.year
df['month'] = pd.to_datetime(df['time'], unit='s').dt.month
df['day'] = pd.to_datetime(df['time'], unit='s').dt.day


In [15]:
# Filtrar Outliers
def filter_outliers(df, tolerance=4):
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
    numeric_cols = numeric_cols.difference(['id', 'price'])
    for col in numeric_cols:
        mean = df[col].mean()
        std = df[col].std()
        lower_bound = mean - (std * tolerance)
        upper_bound = mean + (std * tolerance)
        df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    return df

df_final = filter_outliers(df)
print(df_final.describe())

                 id    bathrooms     bedrooms        price  square_feet  \
count  9.750000e+03  9750.000000  9750.000000  9750.000000  9750.000000   
mean   5.622496e+09     1.345914     1.701765  1416.757538   903.024513   
std    7.068095e+07     0.544479     0.877644   701.641655   407.814262   
min    5.508654e+09     0.491745     0.000000   200.000000   101.000000   
25%    5.509242e+09     1.000000     1.000000   944.000000   647.000000   
50%    5.668609e+09     1.000000     2.000000  1259.000000   800.000000   
75%    5.668626e+09     2.000000     2.000000  1671.500000  1086.750000   
max    5.668662e+09     4.000000     5.000000  5350.000000  2760.000000   

          latitude    longitude          time  price_monthly    year  \
count  9750.000000  9750.000000  9.750000e+03    9750.000000  9750.0   
mean     37.614844   -94.166747  1.574843e+09    1416.757538  2019.0   
std       5.257673    15.110758  3.787602e+06     701.641655     0.0   
min      25.380100  -123.328300  1.5

In [16]:
# Para descargar el excel
df_final.to_excel("datos_apartamentos_rent_actualizado.xlsx", index=False)